**Load Libraries**

In [ ]:
import os
from typing import NoReturn, Any

import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from mlxtend.plotting import plot_confusion_matrix

**Paths**

In [ ]:
BASE_PATH = "../../"
MONITORING = os.path.join(BASE_PATH, "logs")
DATA = os.path.join(BASE_PATH, "data")

In [ ]:
CHECKPOINT_PATH = os.path.join(MONITORING, "checkpoints")
CNN_CHECKPOINT_PATH = os.path.join(CHECKPOINT_PATH, "cnn")
RNN_CHECKPOINT_PATH = os.path.join(CHECKPOINT_PATH, "rnn")

In [ ]:
TENSORBOARD_LOG_DIR = os.path.join(MONITORING, "tensorboard_logs")
CNN_TENSORBOARD_LOGS = os.path.join(TENSORBOARD_LOG_DIR, "cnn")
RNN_TENSORBOARD_LOGS = os.path.join(TENSORBOARD_LOG_DIR, "rnn")

In [ ]:
CSV_LOG_DIR = os.path.join(MONITORING, "csv_logs")
CNN_CSV_LOGS = os.path.join(CSV_LOG_DIR, "cnn")
RNN_CSV_LOGS = os.path.join(CSV_LOG_DIR, "rnn")

In [ ]:
TUNERS = os.path.join(DATA, "tuners")
MODELS = os.path.join(DATA, "models")

**GPU/TPU Multithreading Setup**

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

In [ ]:
if tpu:
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(
        tpu,
    )
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
elif len(gpus) > 1:
    strategy = tf.distribute.MultiWorkerMirroredStrategy([gpu.name for gpu in gpus])
    print("Running on multiple GPUs ", [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy()
    print("Running on single GPU ", gpus[0].name)
else:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")
print("Number of accelerators: ", strategy.num_replicas_in_sync)

**Hyperparameters**

In [ ]:
# Adjustable
BATCH_SIZE = 32  # Big batch size, small learning rate
HEIGHT, WIDTH = 224, 224
IMG_SIZE = (HEIGHT, WIDTH)
IMG_FORMAT = (HEIGHT, WIDTH, 3)
EPOCHS = 1000
TRIALS = 20
SEED = 949953915

**Load Dataset**

In [ ]:
train_dataset, val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../../data/dataset/img/mfcc",
    validation_split=0.2,
    subset="both",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

In [ ]:
class_names = train_dataset.class_names
num_classes = len(class_names)

**Dataset Representation**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

**Preprocessing**

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

In [ ]:
with strategy.scope():    
    normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_ds))

**CNN**

In [ ]:
def cnn(hp: kt.HyperParameters) -> tf.keras.Sequential:
    inputs = tf.keras.Input(shape=IMG_FORMAT)
    x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=3,
        padding="same",
        activation="relu",
        kernel_regularizer=tf.keras.regularizers.l2(0.01),
    )(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(
        filters=hp.Int("conv_2_filter", min_value=64, max_value=256, step=8),
        kernel_size=3,
        padding="same",
        activation="relu",
        kernel_regularizer=tf.keras.regularizers.l2(0.01),
    )(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(
        filters=hp.Int("conv_3_filter", min_value=128, max_value=1024, step=8),
        kernel_size=3,
        padding="same",
        activation="relu",
        kernel_regularizer=tf.keras.regularizers.l2(0.01),
    )(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(
        rate=hp.Float(
            "dropout_1",
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.05,
        )
    )(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(
        units=hp.Int("dense_1_units", min_value=32, max_value=128, step=8),
        activation="relu",
        kernel_regularizer=tf.keras.regularizers.l2(0.01),
    )(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

**Utilitary For Monitoring**

In [ ]:
stop_early = [
    {
        "monitor": "val_loss",
        "patience": 50,
        "min_delta": 0.001,
        "mode": "min",
        "verbose": 1,
        "restore_best_weights": True,
    },
    {
        "monitor": "val_accuracy",
        "patience": 50,
        "min_delta": 0.001,
        "mode": "max",
        "verbose": 1,
        "restore_best_weights": True,
    },
]
stop_early = [tf.keras.callbacks.EarlyStopping(**condition) for condition in stop_early]

In [ ]:
def path_exists(path: str) -> str:
    if os.path.exists(path):
        if path[-1].isdigit():
            suffix = path[: path.rfind("_")]
            digits = int(path[path.rfind("_") + 1 :])
            path = f"{suffix}_{digits + 1}"
        else:
            path = f"{path}_0"
    return path

In [ ]:
def model_checkpoint(model_name):
    return tf.keras.callbacks.ModelCheckpoint(
        filepath=path_exists(
            f"{globals()[f'{model_name.upper()}_CHECKPOINT_PATH']}/"
            f"BS_{BATCH_SIZE}"
            f"_LR_{SEED}"
            f"_EPOCHS_{EPOCHS}"
            f"_TRIALS_{TRIALS}"
        ),
        save_weights_only=False,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
    )

In [ ]:
def tensorboard_logs(model_name: str) -> tf.keras.callbacks.TensorBoard:
    return tf.keras.callbacks.TensorBoard(
        path_exists(
            f"{globals()[f'{model_name.upper()}_TENSORBOARD_LOGS']}/"
            f"BS_{BATCH_SIZE}"
            f"_LR_{SEED}"
            f"_EPOCHS_{EPOCHS}"
            f"_TRIALS_{TRIALS}"
        )
    )

In [ ]:
def epochs_logs(model_name: str) -> tf.keras.callbacks.CSVLogger:
    path = (
        f"{globals()[f'{model_name.upper()}_CSV_LOGS']}/"
        f"BS_{BATCH_SIZE}"
        f"_LR_{SEED}"
        f"_EPOCHS_{EPOCHS}"
        f"_TRIALS_{TRIALS}"
    )
    return tf.keras.callbacks.CSVLogger(f"{path_exists(path)}.csv")

**Training**

In [ ]:
def training(model: Any) -> NoReturn:
    model_name = model.__name__
    with strategy.scope():
        tuner = kt.BayesianOptimization(
            hypermodel=model,
            objective=kt.Objective("val_accuracy", direction="max"),
            max_trials=TRIALS,
            overwrite=True,
            project_name=path_exists(f"{TUNERS}\\{model_name}_tuner"),
            directory=path_exists("{TUNERS}_{model_name}"),
        )

        # Search for best hyperparameters
        tuner.search(
            train_dataset,
            epochs=EPOCHS,
            validation_data=val_dataset,
            callbacks=[
                stop_early,
                model_checkpoint(model_name),
                epochs_logs(model_name),
                tensorboard_logs(model_name),
            ],
        )
        # Get the optimal hyperparameters
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

        # Build model with optimal hyperparameters
        model = tuner.hypermodel.build(best_hps)
        history = model.fit(
            train_dataset,
            epochs=EPOCHS,
            validation_data=val_dataset,
            callbacks=[
                stop_early,
                model_checkpoint(model_name),
                epochs_logs(model_name),
                tensorboard_logs(model_name),
            ],
        )
        val_acc_per_epoch = history.history["val_accuracy"]
        best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
        print(f"best_epoch : {best_epoch}")

        hypermodel = tuner.hypermodel.build(best_hps)
        # Retrain the model with epoch with highest val_accuracy value
        hypermodel.fit(
            train_dataset,
            epochs=best_epoch,
            validation_data=val_dataset,
            callbacks=[
                stop_early,
                model_checkpoint(model_name),
                epochs_logs(model_name),
                tensorboard_logs(model_name),
            ],
        )

        eval_result = hypermodel.evaluate(val_dataset)

        hypermodel.save(
            f"{MODELS}\\"
            f"{model_name}"
            f"_loss_{eval_result[0]}"
            f"_acc_{eval_result[1]}"
            f"_best_epoch_{best_epoch}"
            f"_img_size_{IMG_SIZE}"
        )

In [ ]:
training(cnn)

**Model Evaluation**

In [ ]:
models = [
    f"{root}\\{dir}"
    for root, dirs, files in os.walk(MODELS)
    for dir in dirs
    if "acc" in dir
]
sort_models_per_acc = sorted(
    models,
    key=lambda x: float(
        x[
            x.find("_acc_") + 5 : x.find("_best_")
            if "best" in x
            else x.find("_para_")
            if "_para_" in x
            else None
        ]
    ),
    reverse=True,
)

In [ ]:
best_model = tf.keras.models.load_model(sort_models_per_acc[0])

In [ ]:
sort_models_per_acc[0]

In [ ]:
predictions = best_model.predict(val_dataset).argmax(axis=1)

**Confusion Matrix**

In [ ]:
test_labels = np.concatenate([y for x, y in val_dataset], axis=0)

In [ ]:
confusion_matrix = tf.math.confusion_matrix(test_labels, predictions).numpy()

In [ ]:
fig, ax = plot_confusion_matrix(
    conf_mat=confusion_matrix,
    class_names=val_dataset.class_names,
)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()